# Cocktail party problem

### Preprocessing

In [24]:
path = 'Downloads/'

In [36]:
import IPython.display as ipd
print('Mic1 - Voice')
mic1 = ipd.Audio('rsm2_mA.wav') # load a local WAV file
mic1

Mic1 - Voice


In [37]:
print('Mic2 - Music')
mic2 = ipd.Audio('rsm2_mB.wav') # load a local WAV file
mic2

Mic2 - Music


### Fast ICA for blind source separation

In [38]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from sklearn.decomposition import FastICA, PCA

In [39]:
from scipy.io.wavfile import read
fs1, m1_raw = read("rsm2_mA.wav")
m1 = np.array(m1_raw[1],dtype=float)

In [40]:
fs2, m2_raw = read("rsm2_mB.wav")
m2 = np.array(m2_raw[1], dtype=float)

In [41]:
fs2

16000

In [42]:
S = np.c_[m1, m2]

In [43]:
S.shape

(1, 2)

In [44]:
m1.shape

()

In [45]:
m2.shape

()

In [46]:
S /= S.std(axis=0)  # Standardize data
# Mix data
A = np.array([[1, 1], [0.5, 2]])  # Mixing matrix
X = np.dot(S, A.T)  # Generate observations

# Compute ICA
ica = FastICA(n_components=2)
S_ = ica.fit_transform(X)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

# We can `prove` that the ICA model applies by reverting the unmixing.
assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
pca = PCA(n_components=2)
H = pca.fit_transform(X)  # Reconstruct signals based on orthogonal components

In [ ]:
S_.shape

In [ ]:
plt.figure()

models = [X, S, S_, H]
names = ['Observations (mixed signal)',
         'True Sources',
         'ICA recovered signals',
         'PCA recovered signals']
colors = ['red', 'steelblue', 'orange']

for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(4, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.tight_layout()
plt.show()

In [ ]:
S_.shape

In [ ]:
import sounddevice as sd

#S_ = np.random.uniform(-1, 1, fs1)
ica_recovered_1 = sd.play(S_[:,0], fs1)
ica_recovered_2 = sd.play(S_[:,1], fs2)

In [ ]:
ica_recovered_2

In [ ]:
pca_recovered_1 = sd.play(S_[:,0], fs1)
pca_recovered_2 = sd.play(S_[:,1], fs2)

In [ ]:
pip install sounddevice

## Neural net for blind source separation

In [ ]:
ne